<a href="https://colab.research.google.com/github/kmk4444/Autonomic_agents/blob/main/Part3_openai_assistant_structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- OpenAI created Assistant because of benefit of agents. There are four parts which are Assistant, Thread, Message and Run.
- Assistant means like agent. It is consisted of Name, Instructions and Tools.
- Thanks to thread, we run assistant.
- Then, assistant create message like thread_01.
- Lastly, it added to the end of the chain.

Requirements.txt

In [1]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo langchainhub >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo streamlit >> requirements.txt
!echo duckduckgo-search >> requirements.txt
!echo anthropic >> requirements.txt
!echo beautifulsoup4 >> requirements.txt
!echo autogenstudio >> requirements.txt
!echo crewai >> requirements.txt

terminal / bash komutu

In [ ]:
pip install -r requirements.txt

In [10]:
%%writefile assistant_helper.py

from openai import OpenAI
import time
import os
from dotenv import load_dotenv

#load_dotenv()

#my_key_openai = os.getenv("openai_apikey")
my_key_openai="----"

client = OpenAI(api_key=my_key_openai)

assistant_id = "asst_9PkxJouK0097L2lrUxSznB2U" #| Python Code Assistant

#create a new thread
#create a new run
#create a new message
#take the AI response

#create a new thread
#create a new run
#create a new message
#take the AI response

def start_new_thread():

    thread = client.beta.threads.create()  # This line creates a new thread using the client object.
    thread_id = thread.id # This line retrieves the unique identifier (id) of the newly created thread and stores it in the variable thread_id.

    return thread_id # The function returns the thread_id, which can be used for further interaction with the thread.


def add_message_to_thread(thread_id, prompt):
#This line creates a new message within the thread specified by thread_id. The create method takes several arguments:
    message = client.beta.threads.messages.create(
                thread_id=thread_id, # This specifies the thread where the message will be posted.
                role="user", # This sets the role of the message to "user", indicating it's coming from the user interacting with the AI.
                content=prompt, # This defines the content of the message, which is stored in the prompt variable.
            )


def execute_run_cycle(thread_id):
#This line creates a new "run" within the thread. A "run" might represent a cycle of interaction between the user and the AI. It takes arguments similar to add_message_to_thread:
    run = client.beta.threads.runs.create(
            thread_id=thread_id, # Specifies the thread where the run will occur.
            assistant_id=assistant_id # This variable is likely defined elsewhere and might specify the AI assistant to be used within this run.
        )
#This line starts an infinite loop that continues until the break statement is encountered.
    while True:
#Inside the loop, this line retrieves information about the current run using its thread_id and the previously retrieved run.id.
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        if run.completed_at: #  This line checks if the run has been completed by the AI assistant.
            #If the run is complete, this line calculates the elapsed time between creation and completion.
            elapsed = run.completed_at - run.created_at
            elapsed = time.strftime("%H:%M:%S", time.gmtime(elapsed))
            print(f"Run completed in {elapsed}")
            print("-"*100)
            break
        time.sleep(1)
    #Outside the loop, this line retrieves a list of all messages within the thread.
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    last_message = messages.data[0]

    #This line extracts the text content of the AI's response from the retrieved message object.
    AI_Response = last_message.content[0].text.value

    return AI_Response #The function returns the extracted AI response as text.

Overwriting assistant_helper.py


In [11]:
%%writefile app_assistant.py

import streamlit as st
import assistant_helper
#Checks if "messages" key exists in st.session_state. If not, it creates an empty list for messages and sets thread_id to None. This likely initializes conversation history and thread tracking.
if "messages" not in st.session_state:
    st.session_state.messages = []
    st.session_state.thread_id = None


st.set_page_config(page_title="Python Kodlama Asistanı", page_icon=":speech_balloon:")
st.title("Python Kodlama Asistanı")
st.divider()


for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("Mesajınızı yazınız"):

    with st.chat_message("user"):
        st.markdown(prompt)

#Checks if thread_id is already set in st.session_state. If not:
  #Calls assistant_helper.start_new_thread to initiate a new thread and stores the ID in thread_id.
    with st.spinner("Yanıt oluşturuluyor..."):
        if st.session_state.thread_id is None:
            st.session_state.thread_id = assistant_helper.start_new_thread()
        #Calls assistant_helper.add_message_to_thread to send the user's prompt to the thread.
        assistant_helper.add_message_to_thread(thread_id=st.session_state.thread_id, prompt=prompt)
        #Calls assistant_helper.execute_run_cycle to process the prompt and retrieve the AI response.
        AI_Response = assistant_helper.execute_run_cycle(thread_id=st.session_state.thread_id)

        with st.chat_message("assistant"):
            st.markdown(AI_Response)

    st.session_state.messages.append({"role":"user", "content": prompt})
    st.session_state.messages.append({"role":"assistant", "content": AI_Response})

Overwriting app_assistant.py


In [12]:
!npm install localtunnel
!streamlit run /content/app_assistant.py &>/content/logs.txt &
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.509s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
npx: installed 22 in 2.044s
your url is: https://eleven-trees-thank.loca.lt
^C
